### 판매실적과 차량정보(차종 level)의 수집
- 판매실적: 500개 차종의 2012.11~ 2018.12 월별 판매량
    - requests와 BeautifulSoup 사용
- 차량정보: 500개 차종의  차종, 브랜드, 세그먼트, 출시일, 가격대, 리콜정보
    - selenium  사용

In [1]:
from selenium import webdriver
import os,json,tqdm,pickle, glob,time
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, WebDriverException

from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import re


우선 월별 판매실적을 수집하자

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0'}
params = {'Work':'record'
          ,'Tab':'Model'
          ,'Month':None
          ,'MonthTo':None
          ,'Classify':None}

In [13]:
#페이지 내 판매실적을 가져와 global변수 sold_dic에 쌓는 get_car_sold 함수를 정의
def get_car_sold(soup):
    global sold_dic, exceptions,y,m
    
    # 판매량 테이블 받아온다
    car_solds = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr')
    
    # 브랜드, 세그먼트, 모델, 타이틀, 연월로 구성된 키를 만든다
    # 밸류로 모델판매량(total)과 라인업별 판매량(존재시) 가진다
    for car_sold in car_solds:
        if car_sold.attrs == {}: # model인 경우
            brand = car_sold.find('input')['brand']
            model = car_sold.find('td',{'class':'title'}).find('a')['href'][-4:]
            title = car_sold.find('input')['title']

            key = brand + '_' + params['Classify'] + '_'  + model + '_' + title + '_{year}{month}'.format(year=y,month=m)

            sold_dic[key] = {}
            sold_dic[key]['total'] = int(re.sub('[\D]','',car_sold.find('td',{'class':'num'}).text))

        elif car_sold.attrs['name'] == 'sub_model': # submodel인 경우
            if 'total' in car_sold.attrs['class']:
                continue
            sub_title = car_sold.find('td',{'class':'title'}).text.strip()
            sub_num = int(car_sold.find('td',{'class','num'}).text.replace(',','').strip())
            sold_dic[key][sub_title] = sub_num

        else:
            print('~~!~!')
            pass


In [59]:
if __name__ == '__main__':
    sold_dic = {}
    exceptions = []
    
    # 년월로 쿼리하므로 year, month 를 돌며 url만든후 request
    # 2011.12 ~ 2018.12
    for y in tqdm.tqdm(range(2011,2019)): 
        for m in range(1,13):
            url = 'http://auto.danawa.com/newcar' # base url
            
            if m<10:
                m = str(0)+str(m)
                params['Month'] = '{}-{}-00'.format(y,m)
            else:
                m = str(m)
                params['Month'] = '{}-{}-00'.format(y,m)
                
            
            seg_lst = ['~C','PC1','PC2','PC3','PC4','PC5','P5~','RB~','RU2','RU3','RU5','RM~','O~']
            for seg in seg_lst:
                params['Classify'] = seg
            
                try:
                    res = requests.get(url,params=params,headers=headers,timeout=10)
                    res.encoding = 'utf-8'
                    soup = BeautifulSoup(res.text,'lxml')
                    
                    get_car_sold(soup)
                
                except requests.exceptions.Timeout:
                    exceptions.append((str(y)+str(m),seg))
                    print("Timeout occurred")
                
                except Exception as e:
                    exceptions.append((str(y)+str(m),seg))

In [30]:
# len(sold_dic)

In [ ]:
# len(exceptions)

In [29]:
list(sold_dic.items())[0] # 저장 데이터 예시

('307_~C_3333_모닝_201801', {'total': 4487, '1.0': 227, '1.0  VAN': 205})

In [17]:
# timeout Error 난 파라미터 존재하면 데이터 따로 업데이트 해줌
for e in exceptions:
    y = e[0][:4]
    m = e[0][4:]
    seg = e[1]
    
    url = 'http://auto.danawa.com/newcar'

    params['Month'] = '{}-{}-00'.format(y,m)
    params['Classify'] = seg

    try:
        res = requests.get(url,params=params,headers=headers,timeout=10)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text,'lxml')

        get_car_sold(soup)

    except requests.exceptions.Timeout:
        print(e)
        print("Timeout occurred")

    except Exception as e1:
        print(e1)

In [33]:
# pandas 로 형태로 저장
model_pd = {'brand':[],'seg':[],'model':[],'YM':[],'title':[],'sold':[]} # 차종의 스키마
sub_model_pd =  {'brand':[],'seg':[],'model':[],'YM':[],'title':[],'sub_title':[],'sub_sold':[]} # 서브모델(엔진종류)의 스키마
for k,v in sold_dic.items():
    splited = k.split('_')
    for sub_k,sub_v in v.items():
        if sub_k == 'total':
            continue
        sub_model_pd['brand'].append(splited[0])
        sub_model_pd['seg'].append(splited[1])
        sub_model_pd['model'].append(splited[2])
        sub_model_pd['YM'].append(splited[4])
        sub_model_pd['title'].append(splited[3])
        sub_model_pd['sub_title'].append(sub_k)
        sub_model_pd['sub_sold'].append(sub_v)
        
    model_pd['brand'].append(splited[0])
    model_pd['seg'].append(splited[1])
    model_pd['model'].append(splited[2])
    model_pd['YM'].append(splited[4])
    model_pd['title'].append(splited[3])
    model_pd['sold'].append(v['total'])

mon = pd.DataFrame(model_pd)
mon_sub = pd.DataFrame(sub_model_pd)

In [34]:
mon.head(5) # 차종 단위 판매실적

,brand,seg,model,YM,title,sold
0,307,~C,3333,201801,모닝,4487
1,312,~C,3255,201801,스파크,3347
2,307,~C,3548,201801,레이,2645
3,316,~C,2049,201801,다마스,313
4,316,~C,2052,201801,라보,274


In [35]:
mon_sub.head(5) # 세부 모델(엔진) 단위 판매실적

,brand,seg,model,YM,title,sub_title,sub_sold
0,307,~C,3333,201801,모닝,1.0,227
1,307,~C,3333,201801,모닝,1.0 VAN,205
2,312,~C,3255,201801,스파크,1.0,3347
3,307,~C,3548,201801,레이,1.0,2141
4,307,~C,3548,201801,레이,1.0 LPG,4


In [44]:
# 파일로 출력
os.chdir('./data')
mon_sub.to_excel('monthly_sub.xlsx',encoding='utf-8')
mon.to_excel('monthly.xlsx',encoding='utf-8')

차량 데이터(기본정보) 수집

In [56]:
# car_base의 데이터를 채우는 함수 
def get_car_base(model_id):
    global car_base_dic, params
    
    url = 'http://auto.danawa.com/auto/?attributeList='
    params['Model'] = str(model_id)
    
    for k,v in params.items():
        url += '&'+k+'='+v
    driver.get(url)
    
    # 기본정보 수집
    car_base_dic['model'].append(model_id)
    
    category = driver.find_element_by_css_selector('#autodanawa_gridC > div.gridLeft > ul')
    car_base_dic['brand'].append(category.find_element_by_class_name('depth3').text)
    car_base_dic['seg'].append(category.find_element_by_class_name('depth4').text)
    
    info = driver.find_element_by_class_name('info')
    car_base_dic['title'].append(info.find_element_by_class_name('title').text)
    car_base_dic['launching'].append(info.find_element_by_class_name('date').text)
    
    car_base_dic['price_range'].append(re.sub(r'[가-힣ㄱ-ㅎa-zA-Z|,|\s]','',driver.find_element_by_class_name('price').text.strip()))
    
    #리콜탭
    driver.find_element_by_link_text('보증/리콜').click()
    
    recall_lst = driver.find_elements_by_class_name('recallList')
    car_base_dic['recall_num'].append(len(recall_lst))
    
    recall_txt = []
    for recall in recall_lst:
        recall_subj = recall.find_element_by_class_name('subj')
        recall_txt.append(recall_subj.find_element_by_class_name('cate').text.strip()+'_'+recall_subj.find_element_by_class_name('title').text.strip())
    car_base_dic['recall_txt'].append(recall_txt)

In [68]:
if __name__=='__main__':
    car_base_dic = {'model':[],'brand':[],'seg':[],'title':[],'launching':[],'price_range':[],'recall_num':[],'recall_txt':[]} 
    base_exceptions=[]
    
    # params 재정의
    params = {}
    params['Work'] = 'model'
    params['Model'] = None
    
    # 판매실적 데이터에서 모델아이디를 추출 
    model_ids = [str(el) for el in list(pd.read_excel('monthly.xlsx').model.unique())]
    
    # 크롬드라이버 호출
    os.chdir('./..')
    path = './chromedriver.exe'
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('disable-gpu')
    driver = webdriver.Chrome(path,options=options)
    
    
    # 추출된 차종id에 대해 반복
    for model_id in tqdm.tqdm(model_ids):
        try:
            get_car_base(model_id)
        except Exception as e:
            print(e)
            base_exceptions.append(model_id)
            
    driver.quit()
            
    car_base = pd.DataFrame(car_base_dic)


In [58]:
car_base.head(5)

,model,brand,seg,title,launching,price_range,recall_num,recall_txt
0,3333,기아,경차,모닝,2017.01 ~ 현재,945~1589,0,[]
1,3255,쉐보레,경차,더 넥스트 스파크(M400),2015.08 ~ 2018.04,992~1559,1,[리콜_원동기(엔진)]
2,3548,기아,경차,레이,2017.12 ~ 현재,1210~1670,0,[]
3,2049,쉐보레,상용,다마스,2008.04 ~ 현재,988~1028,0,[]
4,2052,쉐보레,상용,라보,2008.04 ~ 현재,838~1098,0,[]


In [ ]:
# 티볼리 에어 직접 추가
omitted = {'model':['3341',]
           , 'brand':['쌍용',]
           ,'seg':['소형SUV',]
           ,'title':['티볼리 에어',],'duration':['2016.03 ~ 현재',],'price_range':['1866 ~ 2661',], 'recall_num':[0,],'recall_txt':[[]]}
omitted = pd.DataFrame(omitted)
car_base = pd.concat([car_base,omitted],axis=0)

In [ ]:
# 파일출력
car_base.to_excel('car_base.xlsx',encoding='utf-8')

중고차 데이터와 매핑을 위한 중고차 id 역시 수집하자

In [64]:
def get_used_no(model_id):
    global model_used, params
    params['Model'] = str(model_id)
    params['Page'] = None
    
    model_used[str(model_id)] = []
    
    max_page = 20 # 해당 차종의 중고차 상품 몇 페이지 까지 볼건지 
    for p in range(max_page):
        url = 'http://auto.danawa.com/usedcar/?Tab=list'
        params['Page'] = str(p+1)
        for k,v in params.items():
            url += '&'+k +'=' + v
        driver.get(url)
        
        if driver.find_element_by_class_name('total').find_element_by_class_name('num').text == '0': # 해당 페이지에 매물 없으면 break
            break
            
        used_car_lst = driver.find_element_by_id('usedcarList').find_elements_by_class_name('row')

        for used in used_car_lst:
            href = used.find_element_by_class_name('name').get_attribute('href')
            parsed = urlparse(href).query.split('&')
            for kv in [el.split('=') for el in parsed]: # pased 된 리스트에서 sDemoNo 밸류 찾기
                if kv[0] == 'sDemoNo':
                    model_used[model_id].append(kv[1])

In [77]:
if __name__ == '__main__':
    # model과 중고차id를 매핑
    # json 형태로 저장 model_id 가 키
    model_used = {} 
    
    # params 재정의
    params = {}
    params['Model'] = None
    
    # 크롬 드라이버 호출
    os.chdir('./..')
    path = './chromedriver.exe'
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('disable-gpu')
    driver = webdriver.Chrome(path,options=options)

    used_exceptions = []
    for model_id in tqdm.tqdm(model_ids):
        
        try:
            get_used_no(model_id)
            pass
        except Exception as e:
            used_exceptions.append(model_id)
            print(e)
            
    driver.quit()

In [ ]:
# # json으로 출력
# with open('model_used.json','w') as fp:
#     json.dump(model_used)

In [76]:
list(model_used.items())[0]

('3333',
 ['9021100324',
  '9021100309',
  '9021500867',
  'HCL6064',
  '9021500576',
  '9021500522',
  '9021100209',
  'HCL6040',
  '9011002463',
  '9011001268',
  '9011500027',
  '8121002089',
  '94943',
  '94768',
  '8111000470',
  '8101501322',
  '8091501391',
  '8081000803',
  '8071500534',
  '8061100104'])